In [75]:
import pandas as pd

df = pd.read_csv('./data/icu/chartevents.csv', low_memory=False, nrows=1)

df.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,2180-07-23 12:36:00,2180-07-23 14:45:00,226512,39.4,39.4,kg,0


first_day_vitalsign

In [ ]:
"""
---------------------------------
Create first-24-hour vital-sign summary per ICU stay (MIMIC-IV).

• Input  : ./data/icu/icustays.csv
           ./data/icu/chartevents.csv(.gz)

• Output : ./data/icu/first_day_vitalsign.csv
           one row per stay_id, columns:
             heartrate_{min/max/mean}
             sysbp_…  diasbp_…  meanbp_…
             resprate_…  tempc_…  spo2_…  glucose_…

"""

from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

# ------------------------------------------------------------------
# 0.  Paths
# ------------------------------------------------------------------
BASE = Path("./data/icu")          
CHART = BASE / "chartevents.csv"  
ICU   = BASE / "icustays.csv"

# ------------------------------------------------------------------
# 1.  Vital-sign ITEMID → category
# ------------------------------------------------------------------
VITAL_MAP = {
    # Heart rate
    211: 1, 220045: 1,
    # Systolic BP
    51: 2, 442: 2, 455: 2, 6701: 2, 220179: 2, 220050: 2, 225309: 2,
    # Diastolic BP
    8368: 3, 8440: 3, 8441: 3, 8555: 3, 220180: 3, 220051: 3, 225310: 3,
    # Mean BP
    456: 4, 52: 4, 6702: 4, 443: 4, 220052: 4, 220181: 4, 225312: 4,
    # Resp. rate
    615: 5, 618: 5, 220210: 5, 224690: 5,
    # Temperature °F
    223761: 6, 678: 6,
    # Temperature °C
    223762: 6, 676: 6,
    # SpO₂
    646: 7, 220277: 7,
    # Glucose
    807: 8, 811: 8, 1529: 8, 3745: 8, 3744: 8,
    225664: 8, 220621: 8, 226537: 8,
}
TEMP_F_IDS = {223761, 678}         # Fahrenheit → Celsius

CAT2NAME = {
    1: "heartrate", 2: "sysbp", 3: "diasbp", 4: "meanbp",
    5: "resprate", 6: "tempc", 7: "spo2",   8: "glucose",
}

# ------------------------------------------------------------------
# 2.  Load icustays (stay-level times)
# ------------------------------------------------------------------
print("Loading icustays …")
icu = pd.read_csv(ICU, usecols=["stay_id", "subject_id", "hadm_id", "intime"],
                  parse_dates=["intime"])
icu.set_index("stay_id", inplace=True)         # fast lookup
intime_series = icu["intime"]

# ------------------------------------------------------------------
# 3.  Prepare aggregation holders
# ------------------------------------------------------------------
agg_min = {}     # (stay, cat) → min
agg_max = {}     # (stay, cat) → max
agg_sum = {}     # (stay, cat) → running sum
agg_cnt = {}     # (stay, cat) → count for mean

def update(stay, cat, val):
    key = (stay, cat)
    if key not in agg_min:
        agg_min[key] = val
        agg_max[key] = val
        agg_sum[key] = val
        agg_cnt[key] = 1
    else:
        agg_min[key] = min(agg_min[key], val)
        agg_max[key] = max(agg_max[key], val)
        agg_sum[key] += val
        agg_cnt[key] += 1

# ------------------------------------------------------------------
# 4.  Stream chartevents in chunks (huge file)
# ------------------------------------------------------------------
USECOLS = ["stay_id", "itemid", "valuenum", "charttime"]
CHUNK = 1_000_000
print("Scanning chartevents …")
for chunk in pd.read_csv(CHART, usecols=USECOLS,
                         parse_dates=["charttime"],
                         chunksize=CHUNK, low_memory=False):
    # keep rows we might want
    chunk = chunk[chunk["itemid"].isin(VITAL_MAP)]
    # drop junk
    chunk = chunk[pd.notnull(chunk["valuenum"])]

    # attach ICU intime (vectorised map)
    chunk = chunk.join(intime_series, on="stay_id", how="inner")
    # time since admit, in hours
    delta = (chunk["charttime"] - chunk["intime"]).dt.total_seconds() / 3600.0
    chunk = chunk[(delta > 0) & (delta <= 24)]
    if chunk.empty:
        continue

    # Fahrenheit → Celsius
    f_mask = chunk["itemid"].isin(TEMP_F_IDS)
    chunk.loc[f_mask, "valuenum"] = (chunk.loc[f_mask, "valuenum"] - 32.0) / 1.8

    # map to category
    chunk["cat"] = chunk["itemid"].map(VITAL_MAP).astype(np.int8)

    # aggregate
    for row in chunk.itertuples(index=False):
        update(row.stay_id, row.cat, row.valuenum)

# ------------------------------------------------------------------
# 5.  Build wide table
# ------------------------------------------------------------------
print("Building wide DataFrame …")
records = []
for (stay, cat), _ in agg_min.items():
    prefix = CAT2NAME[cat]
    records.append({
        "stay_id": stay,
        f"{prefix}_min" : agg_min[(stay, cat)],
        f"{prefix}_max" : agg_max[(stay, cat)],
        f"{prefix}_mean": agg_sum[(stay, cat)] / agg_cnt[(stay, cat)],
    })
wide = pd.DataFrame(records)

# pivot to one row per stay
wide = wide.groupby("stay_id").first().reset_index()

# add subject/hadm for convenience
wide = wide.merge(icu.reset_index()[["stay_id", "subject_id", "hadm_id"]],
                  on="stay_id", how="left")

# column order
cols = ["subject_id", "hadm_id", "stay_id"]
for name in CAT2NAME.values():
    cols += [f"{name}_min", f"{name}_max", f"{name}_mean"]
wide = wide.reindex(columns=cols)

# ------------------------------------------------------------------
# 6.  Save
# ------------------------------------------------------------------
OUT = BASE / "first_day_vitalsign.csv"
wide.to_csv(OUT, index=False)
print(f"Wrote {len(wide):,} rows to {OUT}")

Loading icustays …
Scanning chartevents …
Building wide DataFrame …
Wrote 73,126 rows to data/icu/first_day_vitalsign.csv


first_day_urine_output.csv

In [ ]:
"""
.py
-------------------------
Sum urine output for the first 24 h of every ICU stay.

• Input  : ./data/icu/icustays.csv(.gz)
           ./data/icu/outputevents.csv(.gz)
• Output : ./data/icu/first_day_urine_output.csv
           columns = subject_id, hadm_id, stay_id, UrineOutput
"""

from pathlib import Path
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import numpy as np

# ------------------------------------------------------------------
# 1.  File locations
# ------------------------------------------------------------------
BASE = Path("./data/icu")         
ICU   = BASE / "icustays.csv"   
OE    = BASE / "outputevents.csv"

# ------------------------------------------------------------------
# 2.  ITEMIDs that represent urine output in MIMIC-IV
# ------------------------------------------------------------------
URINE_IDS = {
    # CareVue
    40055, 43175, 40069, 40094, 40715, 40473, 40085, 40057, 40056,
    40405, 40428, 40086, 40096, 40651,
    # MetaVision
    226559, 226560, 226561, 226584, 226563, 226564,
    226565, 226567, 226557, 226558,
    # GU irrigant (in/out)
    227488, 227489,
}
IRRIGANT_IN = 227488      # subtract this one

# ------------------------------------------------------------------
# 3.  Load ICU stays (gives us intime & patient IDs)
# ------------------------------------------------------------------
print("Loading icustays …")
icu = pd.read_csv(
    ICU,
    usecols=["stay_id", "subject_id", "hadm_id", "intime"],
    parse_dates=["intime"],
)
icu.set_index("stay_id", inplace=True)
intime = icu["intime"]          # Series for fast lookup

# ------------------------------------------------------------------
# 4.  Aggregate urine output in one pass
# ------------------------------------------------------------------
totals = defaultdict(float)      # stay_id → cumulative volume (mL)
USECOLS = ["stay_id", "itemid", "charttime", "value"]
CHUNK = 1_000_000

print("Scanning outputevents …")
for chunk in pd.read_csv(
        OE,
        usecols=USECOLS,
        parse_dates=["charttime"],
        low_memory=False,
        chunksize=CHUNK,
):
    # keep only relevant itemids and non-null values
    chunk = chunk[chunk["itemid"].isin(URINE_IDS)]
    chunk = chunk[pd.notnull(chunk["value"])]
    if chunk.empty:
        continue

    # attach ICU admit time
    chunk = chunk.join(intime, on="stay_id", how="inner")
    # restrict to first 24 h
    delta_hr = (chunk["charttime"] - chunk["intime"]).dt.total_seconds() / 3600.0
    chunk = chunk[(delta_hr > 0) & (delta_hr <= 24)]
    if chunk.empty:
        continue

    # apply GU-irrigant rule (value becomes negative)
    irr_mask = chunk["itemid"] == IRRIGANT_IN
    chunk.loc[irr_mask, "value"] = -1.0 * chunk.loc[irr_mask, "value"]

    # accumulate
    for row in chunk.itertuples(index=False):
        totals[row.stay_id] += row.value

# ------------------------------------------------------------------
# 5.  Build result DataFrame
# ------------------------------------------------------------------
print("Building CSV …")
res = (
    pd.DataFrame({"stay_id": list(totals.keys()),
                  "UrineOutput": list(totals.values())})
      .merge(icu.reset_index()[["stay_id", "subject_id", "hadm_id"]],
             on="stay_id", how="left")
      .loc[:, ["subject_id", "hadm_id", "stay_id", "UrineOutput"]]
      .sort_values(["subject_id", "hadm_id", "stay_id"])
)

OUT = BASE / "first_day_urine_output.csv"
res.to_csv(OUT, index=False)
print(f"Wrote {len(res):,} rows ➜ {OUT}")

Loading icustays …
Scanning outputevents …
Building CSV …
Wrote 69,730 rows ➜ data/icu/first_day_urine_output.csv


first_day_gcs.csv

In [ ]:
#!/usr/bin/env python3
"""
----------------
Compute the minimum Glasgow-Coma-Scale score in the first 24 h of every ICU
stay, following the exact logic in the original SQL.

Input  (./data/icu/)
    • icustays.csv[.gz]         – gives stay_id, subject_id, hadm_id, intime
    • chartevents.csv[.gz]      – raw bedside charting data

Output
    • first_day_gcs.csv         – one row per stay_id with:
        subject_id, hadm_id, stay_id,
        mingcs, gcsmotor, gcsverbal, gcseyes, endotrachflag
"""

from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

# ----------------------------------------------------------------------
# 1.  File locations / parameters
# ----------------------------------------------------------------------
BASE   = Path("./data/icu")          
ICU_FN = BASE / "icustays.csv"    
CE_FN  = BASE / "chartevents.csv"
CHUNK  = 1_000_000                    # streaming

# ----------------------------------------------------------------------
# 2.  ITEMID mapping  (Metavision → CareVue IDs)
# ----------------------------------------------------------------------
MAP_IDS = {
    223900: 723,   # Verbal
    223901: 454,   # Motor
    220739: 184,   # Eyes
}
ALL_IDS = {184, 454, 723, *MAP_IDS.keys()}

TEMP_INTUB_IDS = {   # (itemid, value_text) pairs that mean “intubated”
    (723,    "1.0 ET/Trach"),
    (223900, "No Response-ETT"),
}

# ----------------------------------------------------------------------
# 3.  Load icustays (intime & identifiers)
# ----------------------------------------------------------------------
print("Loading icustays …")
icu = pd.read_csv(
    ICU_FN,
    usecols=["stay_id", "subject_id", "hadm_id", "intime"],
    parse_dates=["intime"],
)
icu.set_index("stay_id", inplace=True)
intime = icu["intime"]                       # Series for fast lookup

# ----------------------------------------------------------------------
# 4.  Holders for per-stay minimum GCS info
# ----------------------------------------------------------------------
min_gcs      = {}          # stay_id → minimal GCS score
min_components = {}        # stay_id → (motor, verbal, eyes, endotrachflag)

# For each stay we also track the “previous” GCS components (<=6 h old)
prev_val   = {}            # stay_id → dict(component → (value, time))

def update_prev(stay, comp, val, t):
    """Store the latest component value for a stay (used for ≤6 h look-back)."""
    if stay not in prev_val:
        prev_val[stay] = {}
    prev_val[stay][comp] = (val, t)

def get_prev(stay, comp, now):
    """Return previous value of <comp> if within 6 h, else None."""
    if stay not in prev_val or comp not in prev_val[stay]:
        return None
    val, t_prev = prev_val[stay][comp]
    if (now - t_prev).total_seconds() / 3600.0 <= 6:
        return val
    return None

# ----------------------------------------------------------------------
# 5.  Stream chartevents and process in time order
# ----------------------------------------------------------------------
USECOLS = ["stay_id", "itemid", "charttime", "valuenum", "value"]
print("Scanning chartevents …")
for chunk in pd.read_csv(
        CE_FN,
        usecols=USECOLS,
        parse_dates=["charttime"],
        chunksize=CHUNK,
        low_memory=False):

    # keep only rows we want
    chunk = chunk[chunk["itemid"].isin(ALL_IDS)]
    chunk = chunk.merge(intime, on="stay_id", how="inner")      # add ICU intime
    # first-24-h window
    delta = (chunk["charttime"] - chunk["intime"]).dt.total_seconds() / 3600.0
    chunk = chunk[(delta > 0) & (delta <= 24)]
    if chunk.empty:
        continue

    # map Metavision IDs → CareVue IDs
    chunk["itemid"] = chunk["itemid"].replace(MAP_IDS)

    # figure out numeric component value
    def _num(row):
        # Intubated verbal response → 0
        if (row.itemid, str(row.value).strip()) in TEMP_INTUB_IDS:
            return 0.0
        return row.valuenum
    chunk["comp_val"] = chunk.apply(_num, axis=1)

    # drop rows with no numeric value
    chunk = chunk[pd.notnull(chunk["comp_val"])]

    # sort so we visit events in chronological order per stay
    chunk.sort_values(["stay_id", "charttime"], inplace=True)

    # process rows
    for row in chunk.itertuples(index=False):
        stay   = row.stay_id
        comp   = row.itemid          # 454 motor, 723 verbal, 184 eyes
        val    = float(row.comp_val)
        tstamp = row.charttime

        # record component value for this timestamp
        update_prev(stay, comp, val, tstamp)

        # Get current set of components (some may be None)
        motor  = val if comp == 454 else get_prev(stay, 454, tstamp)
        verbal = val if comp == 723 else get_prev(stay, 723, tstamp)
        eyes   = val if comp == 184 else get_prev(stay, 184, tstamp)

        # We only compute if at least one component is new/non-null
        if motor is None and verbal is None and eyes is None:
            continue

        endotrachflag = 1 if verbal == 0 else 0

        # previous (≤6 h) components for imputation
        pmotor  = get_prev(stay, 454, tstamp)
        pverbal = get_prev(stay, 723, tstamp)
        peyes   = get_prev(stay, 184, tstamp)

        # ------- SQL logic replicated -------
        if verbal == 0 or (verbal is None and pverbal == 0):
            gcs = 15
        elif pverbal == 0:
            gcs = (
                (motor  if motor  is not None else 6) +
                (verbal if verbal is not None else 5) +
                (eyes   if eyes   is not None else 4)
            )
        else:
            gcs = (
                (motor  if motor  is not None else (pmotor  if pmotor  is not None else 6)) +
                (verbal if verbal is not None else (pverbal if pverbal is not None else 5)) +
                (eyes   if eyes   is not None else (peyes   if peyes   is not None else 4))
            )
        # ------------------------------------

        # keep minimal GCS
        if (stay not in min_gcs) or (gcs < min_gcs[stay]):
            min_gcs[stay]        = gcs
            min_components[stay] = (
                motor  if motor  is not None else pmotor  if pmotor  is not None else 6,
                verbal if verbal is not None else pverbal if pverbal is not None else 5,
                eyes   if eyes   is not None else peyes   if peyes   is not None else 4,
                endotrachflag,
            )

# ----------------------------------------------------------------------
# 6.  Assemble result table
# ----------------------------------------------------------------------
print("Building result …")
records = []
for stay, gcs in min_gcs.items():
    motor, verbal, eyes, etflag = min_components[stay]
    subj  = icu.at[stay, "subject_id"]
    hadm  = icu.at[stay, "hadm_id"]
    records.append(
        (subj, hadm, stay, gcs, motor, verbal, eyes, etflag)
    )

res = pd.DataFrame(
    records,
    columns=[
        "subject_id", "hadm_id", "stay_id",
        "mingcs", "gcsmotor", "gcsverbal", "gcseyes", "endotrachflag"
    ],
).sort_values(["subject_id", "hadm_id", "stay_id"])

# ----------------------------------------------------------------------
# 7.  Save
# ----------------------------------------------------------------------
OUT = BASE / "first_day_gcs.csv"
res.to_csv(OUT, index=False)
print(f"Wrote {len(res):,} rows ➜ {OUT}")

Loading icustays …
Scanning chartevents …
Building result …
Wrote 72,538 rows ➜ data/icu/first_day_gcs.csv


first_day_lab.csv

In [ ]:
"""
================
Build first_day_lab.csv from local MIMIC-IV CSVs.

Input  (./data/)
    • icu/icustays.csv[.gz]       – stay_id, subject_id, hadm_id, intime
    • hosp/labevents.csv[.gz]     – lab results

Output
    • icu/first_day_lab.csv
      columns:
        subject_id, hadm_id, stay_id,
        aniongap_min, aniongap_max, albumin_min, albumin_max, …, wbc_max
"""

from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

# ----------------------------------------------------------------------
# 1  Paths & parameters
# ----------------------------------------------------------------------
ROOT  = Path("./data")
ICU   = ROOT / "icu"  / "icustays.csv"
LAB   = ROOT / "hosp" / "labevents.csv"
CHUNK = 1_000_000                       # rows streamed at once

# ----------------------------------------------------------------------
# 2  ItemID → label mapping  (unchanged from official SQL)
# ----------------------------------------------------------------------
ID2LAB = {
    50868: "aniongap",
    50862: "albumin",
    51144: "bands",
    50882: "bicarbonate",
    50885: "bilirubin",
    50912: "creatinine",
    50902: "chloride",
    50806: "chloride",
    50809: "glucose",
    50931: "glucose",
    50810: "hematocrit",
    51221: "hematocrit",
    50811: "hemoglobin",
    51222: "hemoglobin",
    50813: "lactate",
    51265: "platelet",
    50822: "potassium",
    50971: "potassium",
    51275: "ptt",
    51237: "inr",
    51274: "pt",
    50824: "sodium",
    50983: "sodium",
    51006: "bun",
    51301: "wbc",
    51300: "wbc",
}

# sanity-check upper limits (itemid → max allowed); values above are discarded
UPPER = {
    50862: 10,     50868: 1e4,  51144: 100,
    50882: 1e4,    50885: 150,  50912: 150,
    50806: 1e4,    50902: 1e4,  50809: 1e4,
    50931: 1e4,    50810: 100,  51221: 100,
    50811: 50,     51222: 50,   50813: 50,
    51265: 1e4,    50822: 30,   50971: 30,
    51275: 150,    51237: 50,   51274: 150,
    50824: 200,    50983: 200,  51006: 300,
    51300: 1000,   51301: 1000,
}

KEEP_IDS = set(ID2LAB.keys())

LABS = ["aniongap","albumin","bands","bicarbonate","bilirubin","creatinine",
        "chloride","glucose","hematocrit","hemoglobin","lactate","platelet",
        "potassium","ptt","inr","pt","sodium","bun","wbc"]

# ----------------------------------------------------------------------
# 3  Load icustays (small) – gives intime and IDs
# ----------------------------------------------------------------------
print("Loading icustays …")
icu = pd.read_csv(
    ICU,
    usecols=["stay_id", "subject_id", "hadm_id", "intime"],
    parse_dates=["intime"],
)
icu.set_index("stay_id", inplace=True)
intime = icu["intime"]                      # Series for quick lookup

# ----------------------------------------------------------------------
# 4  Dictionaries to collect min / max per stay & lab
# ----------------------------------------------------------------------
lab_min = {}      # (stay, lab) → min value
lab_max = {}      # (stay, lab) → max value

def update(stay, lab, val):
    key = (stay, lab)
    if key not in lab_min:
        lab_min[key] = lab_max[key] = val
    else:
        lab_min[key] = min(lab_min[key], val)
        lab_max[key] = max(lab_max[key], val)

# ----------------------------------------------------------------------
# 5  Stream labevents
# ----------------------------------------------------------------------
print("Scanning labevents …")
USE = ["subject_id","hadm_id","itemid","charttime","valuenum"]
for chunk in pd.read_csv(
        LAB, usecols=USE, parse_dates=["charttime"],
        chunksize=CHUNK, low_memory=False):

    chunk = chunk[chunk["itemid"].isin(KEEP_IDS)]
    chunk = chunk[pd.notnull(chunk["valuenum"]) & (chunk["valuenum"] > 0)]
    if chunk.empty:
        continue

    # merge to icustays via (subject_id, hadm_id)
    chunk = chunk.merge(
        icu.reset_index()[["stay_id","subject_id","hadm_id","intime"]],
        on=["subject_id","hadm_id"], how="inner",
    )
    if chunk.empty:
        continue

    # time filter: intime-6 h ≤ charttime ≤ intime+24 h
    dt = (chunk["charttime"] - chunk["intime"]).dt.total_seconds() / 3600.0
    chunk = chunk[(dt >= -6) & (dt <= 24)]
    if chunk.empty:
        continue

    # quality filters
    mask_good = np.ones(len(chunk), dtype=bool)
    for item, lim in UPPER.items():
        idx = (chunk["itemid"] == item) & (chunk["valuenum"] > lim)
        mask_good &= ~idx
    # bands allow 0–100 only
    mask_good &= ~((chunk["itemid"] == 51144) & (chunk["valuenum"] < 0))
    chunk = chunk[mask_good]
    if chunk.empty:
        continue

    # map itemid → lab label
    chunk["lab"] = chunk["itemid"].map(ID2LAB)

    # aggregate
    for row in chunk.itertuples(index=False):
        update(row.stay_id, row.lab, row.valuenum)

# ----------------------------------------------------------------------
# 6  Build wide table
# ----------------------------------------------------------------------
print("Building wide DataFrame …")
records = []
for (stay, lab) in lab_min:
    records.append({
        "stay_id": stay,
        f"{lab}_min": lab_min[(stay, lab)],
        f"{lab}_max": lab_max[(stay, lab)],
    })
wide = pd.DataFrame(records).groupby("stay_id").first().reset_index()

# add identifiers
wide = wide.merge(
    icu.reset_index()[["stay_id","subject_id","hadm_id"]],
    on="stay_id", how="left",
)

# column order
cols = ["subject_id","hadm_id","stay_id"]
for lb in LABS:
    cols += [f"{lb}_min", f"{lb}_max"]
wide = wide.reindex(columns=cols)

# ----------------------------------------------------------------------
# 7  Save
# ----------------------------------------------------------------------
OUT = ROOT / "icu" / "first_day_lab.csv"
wide.to_csv(OUT, index=False)
print(f"Wrote {len(wide):,} rows ➜ {OUT}")

Loading icustays …
Scanning labevents …
Building wide DataFrame …
Wrote 71,945 rows ➜ data/icu/first_day_lab.csv


first_day_blood_gas.csv

In [ ]:
"""
first_day_blood_gas.py
----------------------
Pivot all blood-gas / chemistry results taken in the first 24 h of each ICU
stay into one wide table (one row per stay+chart-time).

Inputs   ./data/icu/
    • icustays.csv[.gz]         – stay_id, subject_id, hadm_id, intime
    • ../hosp/labevents.csv[.gz]  (or same folder) – raw lab events

Output   ./data/icu/first_day_blood_gas.csv
"""

from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl

# ----------------------------------------------------------------------
# 1.  File locations (edit if your layout differs)
# ----------------------------------------------------------------------
ROOT   = Path("./data")
ICU_F  = ROOT / "icu"  / "icustays.csv"
LAB_F  = ROOT / "hosp" / "labevents.csv"     # typical kaggle export
        
OUT_F  = ROOT / "icu" / "first_day_blood_gas.csv"
CHUNK  = 1_000_000

# ----------------------------------------------------------------------
# 2.  ITEMID → label mapping  (exact copy of the SQL CASE block)
# ----------------------------------------------------------------------
ID2LBL = {
    50800:'specimen', 50801:'aado2', 50802:'baseexcess', 50803:'bicarbonate',
    50804:'totalco2', 50805:'carboxyhemoglobin', 50806:'chloride',
    50808:'calcium',   50809:'glucose', 50810:'hematocrit', 50811:'hemoglobin',
    50812:'intubated', 50813:'lactate', 50814:'methemoglobin', 50815:'o2flow',
    50816:'fio2',      50817:'so2',      50818:'pco2', 50819:'peep',
    50820:'ph',        50821:'po2',      50822:'potassium', 50823:'requiredo2',
    50824:'sodium',    50825:'temperature', 50826:'tidalvolume',
    50827:'ventilationrate', 50828:'ventilator',
}

KEEP_IDS = set(ID2LBL.keys()) | {51545}      # 51545 = “something” in old view
LABELS = list(dict.fromkeys(ID2LBL.values()))  # keep order, remove dup

# upper-limit sanity filters (same as SQL WHEN … THEN NULL)
UPPER = {50810:100, 50816:100, 50817:100, 50815:70, 50821:800}

# ----------------------------------------------------------------------
# 3.  Load icustays (small)
# ----------------------------------------------------------------------
print("Loading icustays …")
icu = pd.read_csv(ICU_F,
                  usecols=["stay_id","subject_id","hadm_id","intime"],
                  parse_dates=["intime"])
icu.set_index("stay_id", inplace=True)
intime = icu["intime"]

# ----------------------------------------------------------------------
# 4.  Streaming pivot
# ----------------------------------------------------------------------
print("Scanning labevents …")
rows = []

usecols = ["subject_id","hadm_id",
           "itemid","charttime","value","valuenum"]
for chunk in pd.read_csv(LAB_F, usecols=usecols, parse_dates=["charttime"],
                         chunksize=CHUNK, low_memory=False):

    chunk = chunk[chunk["itemid"].isin(KEEP_IDS)]
    chunk = chunk[pd.notnull(chunk["valuenum"])]
    if chunk.empty:
        continue

    # joinOn stay_id quickly via subject+hadm
    chunk = chunk.merge(icu.reset_index(),
                        on=["subject_id","hadm_id"], how="inner")
    if chunk.empty:
        continue

    # first-day window  [-6h, +24h]
    dt = (chunk["charttime"] - chunk["intime"]).dt.total_seconds()/3600
    chunk = chunk[(dt >= -6) & (dt <= 24)]
    if chunk.empty:
        continue

    # sanity truncations
    ok = np.ones(len(chunk), bool)
    for iid, limit in UPPER.items():
        ok &= ~((chunk["itemid"] == iid) & (chunk["valuenum"] > limit))
    chunk = chunk[ok]
    if chunk.empty:
        continue

    # FiO2 must lie 21-100
    mask_fio2 = chunk["itemid"] == 50816
    chunk.loc[mask_fio2 & (chunk["valuenum"] < 20), "valuenum"] = np.nan
    chunk.loc[mask_fio2 & (chunk["valuenum"] > 100), "valuenum"] = np.nan

    # O2 sat ≤ 100
    mask_so2 = chunk["itemid"] == 50817
    chunk.loc[mask_so2 & (chunk["valuenum"] > 100), "valuenum"] = np.nan

    # negative values not allowed except BASEEXCESS (50802)
    neg_mask = (chunk["valuenum"] <= 0) & (chunk["itemid"] != 50802)
    chunk.loc[neg_mask, "valuenum"] = np.nan

    # map to short label
    chunk["label"] = chunk["itemid"].map(ID2LBL)

    # keep shortest representation of specimen (text “value” column)
    chunk.loc[chunk["label"] == "specimen", "valuenum"] = np.nan

    rows.append(chunk[["stay_id","charttime","label","value","valuenum"]])

if not rows:
    raise RuntimeError("No blood-gas rows found!")

df = pd.concat(rows, ignore_index=True)

# ----------------------------------------------------------------------
# 5.  Pivot to wide format
# ----------------------------------------------------------------------
print("Pivoting …")
wide = (
    df.pivot_table(index=["stay_id","charttime"],
                   columns="label",
                   values="valuenum",
                   aggfunc="max")         # duplicates same chart-time → max()
      .reset_index()
)

# specimen is textual → pull from original rows via first non-null value
spec = (df[df["label"]=="specimen"]
        .drop_duplicates(subset=["stay_id","charttime"])
        .set_index(["stay_id","charttime"])["value"])
wide["specimen"] = wide.set_index(["stay_id","charttime"]).index.map(spec)

# add subject_id / hadm_id
wide = wide.merge(icu.reset_index()[["stay_id","subject_id","hadm_id"]],
                  on="stay_id", how="left")

# column order: IDs, charttime, specimen, then every lab
cols = ["subject_id","hadm_id","stay_id","charttime","specimen"]
for lb in LABELS:
    cols.append(lb)
wide = wide.reindex(columns=cols)

# ----------------------------------------------------------------------
# 6.  Save
# ----------------------------------------------------------------------
print("Writing CSV …")
wide.to_csv(OUT_F, index=False)
print(f"Wrote {len(wide):,} rows ➜ {OUT_F}")

Loading icustays …
Scanning labevents …
Pivoting …
Writing CSV …
Wrote 222,866 rows ➜ data/icu/first_day_blood_gas.csv


first_day_bg_art.csv

In [66]:
#!/usr/bin/env python
# extract_bg_oxygen.py
# Build an arterial-blood-gas view with inferred specimen type (ART)
# using MIMIC-IV (v2.2) CSVs.

import pathlib, numpy as np, pandas as pd
import polars as pl
import pyarrow
from math import exp

DATA_DIR = pathlib.Path("./data")
ICU_DIR  = DATA_DIR / "icu"

# ───────────────────────────────
# 1. Load the raw files we need
# ───────────────────────────────
print("Loading chartevents …")
ce = pd.read_csv(
    ICU_DIR / "chartevents.csv",
    usecols=[
        "subject_id", "hadm_id", "stay_id",
        "charttime", "itemid", "valuenum"
    ],
    dtype={
        "subject_id": "int32",
        "hadm_id":    "Int32",
        "stay_id":    "int32",
        "itemid":     "int32",
        "valuenum":   "float32",
    },
    parse_dates=["charttime"]
)

print("Loading first-day blood-gases …")
bg = pd.read_csv(
    ICU_DIR / "first_day_blood_gas.csv",
    parse_dates=["charttime"]
)
# stay_id in MIMIC-IV == icustay_id in the original query
bg["stay_id"] = bg["stay_id"].astype("int32")
# ───────────────────────────────
# 2. Stage 1 ─ SpO₂
# ───────────────────────────────
SPO2_ITEMS = [646, 220277]   # same codes in MIMIC-IV

spo2 = (
    ce.loc[ce.itemid.isin(SPO2_ITEMS)]
      .assign(valuenum=lambda d: d.valuenum.where(
          (d.valuenum > 0) & (d.valuenum <= 100)
      ))
      .groupby(["subject_id", "hadm_id", "stay_id", "charttime"], as_index=False)
      .agg(SpO2=("valuenum", "max"))
)

# ───────────────────────────────
# 3. Stage 1 ─ FiO₂
# ───────────────────────────────
FIO2_ITEMS = [3420, 3422, 190, 223835]

def _clean_fio2(row):
    v = row.valuenum
    if row.itemid == 223835:              # Inspired O₂ Fraction
        if 0 < v <= 1:   return v * 100
        if 21 <= v <= 100: return v
    elif row.itemid in (3420, 3422):      # well-formatted %
        return v
    elif row.itemid == 190:               # 0-1 range
        if 0.20 < v < 1: return v * 100
    return np.nan

fio2_raw = ce.loc[ce.itemid.isin(FIO2_ITEMS)].copy()
fio2_raw["fio2_clean"] = fio2_raw.apply(_clean_fio2, axis=1)

fio2 = (
    fio2_raw
      .groupby(["subject_id", "hadm_id", "stay_id", "charttime"], as_index=False)
      .agg(fio2_chartevents=("fio2_clean", "max"))
)

# ───────────────────────────────
# 4. Attach the most recent SpO₂ ≤ 2 h
# ───────────────────────────────
print("Merging SpO₂ …")

bg_pl = pl.from_pandas(bg)
spo2_pl = (
    pl.read_csv(
        "./data/icu/chartevents.csv",
        columns=["subject_id","hadm_id","stay_id","charttime","itemid","valuenum"],
        try_parse_dates=True
    )
    .filter(pl.col("itemid").is_in([646,220277]) &
            (pl.col("valuenum") > 0) & (pl.col("valuenum") <= 100))
    .select([
        "subject_id","hadm_id","stay_id",
        pl.col("charttime").alias("charttime"),
        pl.col("valuenum").alias("Spo2")
    ])
)

bg_pl   = bg_pl.with_columns(pl.col("stay_id").cast(pl.Int32))
spo2_pl = spo2_pl.with_columns(pl.col("stay_id").cast(pl.Int32))

TIME_UNIT = "us"  # polars default is nanoseconds, we want microseconds 
bg_pl   = bg_pl.with_columns(
             pl.col("charttime").dt.cast_time_unit(TIME_UNIT)
         )
spo2_pl = spo2_pl.with_columns(
             pl.col("charttime").dt.cast_time_unit(TIME_UNIT)
         )

result_pl = (
    bg_pl.join_asof(
        spo2_pl,
        on="charttime",
        by="stay_id",
        strategy="backward",
        tolerance="2h"))

print(result_pl.head())
# ───────────────────────────────
# 5. Attach the most recent FiO₂ ≤ 4 h
# ───────────────────────────────
# ── Build FiO₂ table and attach it (≤ 4 h backward) ───────────────
FIO2_ITEMS = [3420, 3422, 190, 223835]

fio2_pl = (
    pl.read_csv(
        "./data/icu/chartevents.csv",
        columns=["stay_id", "charttime", "itemid", "valuenum"],
        try_parse_dates=True,
    )
    # keep only FiO₂ rows
    .filter(pl.col("itemid").is_in(FIO2_ITEMS))
    # apply the cleansing rules -> numeric %
    .with_columns(
        (
            pl.when(pl.col("itemid") == 223835)
              .then(
                  pl.when((pl.col("valuenum") > 0) & (pl.col("valuenum") <= 1))
                    .then(pl.col("valuenum") * 100)
                  .when((pl.col("valuenum") >= 21) & (pl.col("valuenum") <= 100))
                    .then(pl.col("valuenum"))
                  .otherwise(None)
              )
            .when(pl.col("itemid").is_in([3420, 3422]))
              .then(pl.col("valuenum"))
            .when(
                (pl.col("itemid") == 190)
                & (pl.col("valuenum") > 0.20)
                & (pl.col("valuenum") < 1)
            )
              .then(pl.col("valuenum") * 100)
            .otherwise(None)
            .cast(pl.Float32)
            .alias("fio2_ce")
        )
    )
    # one FiO₂ per stay_id-charttime → take the max
    .group_by(["stay_id", "charttime"])
    .agg(pl.col("fio2_ce").max())
    # keep only join keys + value
    .select(["stay_id", "charttime", "fio2_ce"])
    # match dtypes with result_pl
    .with_columns(
        [
            pl.col("stay_id").cast(pl.Int32),
            pl.col("charttime").dt.cast_time_unit("us"),
        ]
    )
    .drop_nulls(["stay_id", "charttime"])
    .sort(["stay_id", "charttime"])        # required for join_asof
)

# ensure left frame keys are sorted & typed
result_pl = (
    result_pl
    .with_columns(
        [
            pl.col("stay_id").cast(pl.Int32),
            pl.col("charttime").dt.cast_time_unit("us"),
        ]
    )
    .sort(["stay_id", "charttime"])
)

# as-of join: latest FiO₂ ≤ 4 h before the blood gas
result_pl = (
    result_pl
    .join_asof(
        fio2_pl,
        by="stay_id",
        on="charttime",
        strategy="backward",
        tolerance="4h",
    )
    .rename({"fio2_ce": "fio2_chartevents"})
)

print(result_pl.head())
# ───────────────────────────────
# 6. Compute SPECIMEN_PROB
# ───────────────────────────────
coef  = {
    "intercept": -0.02544,
    "po2":        0.04598,
    "spo2":      -0.15356,
    "fio2_ce":    0.00621,
    "hemoglobin": 0.10559,
    "so2":        0.13251,
    "pco2":      -0.01511,
    "fio2":       0.01480,
    "aado2":     -0.00200,
    "bicarbonate":-0.03220,
    "totalco2":   0.05384,
    "lactate":    0.08202,
    "ph":         0.10956,
    "o2flow":     0.00848,
}
mean  = {
    "spo2":        97.49420,
    "fio2_ce":     51.49550,
    "hemoglobin":  10.32307,
    "so2":         93.66539,
    "pco2":        42.08866,
    "fio2":        63.97836,
    "aado2":      442.21186,
    "bicarbonate": 22.96894,
    "totalco2":    24.72632,
    "lactate":      3.06436,
    "ph":           7.36233,
    "o2flow":       7.59362,
}

def logistic_expr(expr: pl.Expr) -> pl.Expr:
    """Polars-friendly logistic."""
    return 1 / (1 + (-expr).exp())

result_pl = (
    result_pl
    .with_columns([
        # replace NaNs with hard-coded means
        pl.col("Spo2").fill_null(mean["spo2"]).alias("_spo2"),
        pl.col("fio2_chartevents").fill_null(mean["fio2_ce"]).alias("_fio2_ce"),
        pl.col("hemoglobin").fill_null(mean["hemoglobin"]).alias("_hgb"),
        pl.col("so2").fill_null(mean["so2"]).alias("_so2"),
        pl.col("pco2").fill_null(mean["pco2"]).alias("_pco2"),
        pl.col("fio2").fill_null(mean["fio2"]).alias("_fio2"),
        pl.col("aado2").fill_null(mean["aado2"]).alias("_aado2"),
        pl.col("bicarbonate").fill_null(mean["bicarbonate"]).alias("_bicarb"),
        pl.col("totalco2").fill_null(mean["totalco2"]).alias("_tco2"),
        pl.col("lactate").fill_null(mean["lactate"]).alias("_lact"),
        pl.col("ph").fill_null(mean["ph"]).alias("_ph"),
        pl.col("o2flow").fill_null(mean["o2flow"]).alias("_o2flow"),
    ])
    .with_columns([
        (
            coef["intercept"]
            + coef["po2"]        * pl.col("po2")
            + coef["spo2"]       * pl.col("_spo2")          + 0.13429
            + coef["fio2_ce"]    * pl.col("_fio2_ce")       - 0.24958
            + coef["hemoglobin"] * pl.col("_hgb")           + 0.05954
            + coef["so2"]        * pl.col("_so2")           - 0.23172
            + coef["pco2"]       * pl.col("_pco2")          - 0.01630
            + coef["fio2"]       * pl.col("_fio2")          - 0.31142
            + coef["aado2"]      * pl.col("_aado2")         - 0.01328
            + coef["bicarbonate"]* pl.col("_bicarb")        - 0.06535
            + coef["totalco2"]   * pl.col("_tco2")          - 0.01405
            + coef["lactate"]    * pl.col("_lact")          + 0.06038
            + coef["ph"]         * pl.col("_ph")            - 0.00617
            + coef["o2flow"]     * pl.col("_o2flow")        - 0.35803
        ).alias("_z")
    ])
    .with_columns([
        logistic_expr(pl.col("_z")).alias("SPECIMEN_PROB")
    ])
)

# ───────────────────────────────
# 7. Derived metrics
# ───────────────────────────────
result_pl = (
    result_pl
    .with_columns([
        # A–a DO2
        pl.when(
            pl.all_horizontal(
                pl.col("po2").is_not_null(),
                pl.col("pco2").is_not_null(),
                (pl.col("fio2").is_not_null() | pl.col("fio2_chartevents").is_not_null())
            )
        ).then(
            (pl.coalesce([pl.col("fio2"), pl.col("fio2_chartevents")]) / 100)
            * (760 - 47)
            - (pl.col("pco2") / 0.8)
            - pl.col("po2")
        ).otherwise(None).alias("AADO2_calc"),

        # PaO2 / FiO2
        pl.when(
            pl.col("po2").is_not_null() &
            (pl.col("fio2").is_not_null() | pl.col("fio2_chartevents").is_not_null())
        ).then(
            100 * pl.col("po2") /
            pl.coalesce([pl.col("fio2"), pl.col("fio2_chartevents")])
        ).otherwise(None).alias("PaO2FiO2")
    ])
)

# ───────────────────────────────
# 8. Keep arterial samples only
# ───────────────────────────────
result_pl = result_pl.with_columns(
    pl.col("specimen").cast(pl.Utf8)
)
result_pl = (
    result_pl
    .filter(
        (pl.col("specimen") == "ART") |
        (pl.col("SPECIMEN_PROB") > 0.75)
    )
    .sort(["stay_id", "charttime"])
)

# ───────────────────────────────
# 9. Save
# ───────────────────────────────
result_pl.write_csv("first_day_bg_art.csv")
print(f"Done. {result_pl.height:,} rows written to first_day_bg_art.csv")

Loading chartevents …
Loading first-day blood-gases …
Merging SpO₂ …


/var/folders/cm/blwm27yd02l6xpvwrm9_l41m0000gn/T/ipykernel_36181/3513169847.py:113: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  bg_pl.join_asof(


shape: (5, 36)
┌────────────┬──────────┬──────────┬────────────┬───┬────────────┬────────────┬────────────┬───────┐
│ subject_id ┆ hadm_id  ┆ stay_id  ┆ charttime  ┆ … ┆ ventilator ┆ subject_id ┆ hadm_id_ri ┆ Spo2  │
│ ---        ┆ ---      ┆ ---      ┆ ---        ┆   ┆ ---        ┆ _right     ┆ ght        ┆ ---   │
│ i64        ┆ i64      ┆ i32      ┆ datetime[μ ┆   ┆ f64        ┆ ---        ┆ ---        ┆ f64   │
│            ┆          ┆          ┆ s]         ┆   ┆            ┆ i64        ┆ i64        ┆       │
╞════════════╪══════════╪══════════╪════════════╪═══╪════════════╪════════════╪════════════╪═══════╡
│ 12466550   ┆ 23998182 ┆ 30000153 ┆ 2174-09-29 ┆ … ┆ null       ┆ 12466550   ┆ 23998182   ┆ 100.0 │
│            ┆          ┆          ┆ 13:27:00   ┆   ┆            ┆            ┆            ┆       │
│ 12466550   ┆ 23998182 ┆ 30000153 ┆ 2174-09-29 ┆ … ┆ null       ┆ 12466550   ┆ 23998182   ┆ 100.0 │
│            ┆          ┆          ┆ 14:07:00   ┆   ┆            ┆          

/var/folders/cm/blwm27yd02l6xpvwrm9_l41m0000gn/T/ipykernel_36181/3513169847.py:190: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  .join_asof(


In [54]:
print("fio2_pl schema:", fio2_pl.schema)   # OR

print("result_pl schema:", result_pl.schema)  # OR


fio2_pl schema: Schema([('stay_id', Int32), ('charttime', Datetime(time_unit='us', time_zone=None)), ('fio2_ce', Float32)])
result_pl schema: Schema([('subject_id', Int64), ('hadm_id', Int64), ('stay_id', Int32), ('charttime', Datetime(time_unit='us', time_zone=None)), ('specimen', Float64), ('specimen.1', Float64), ('aado2', Float64), ('baseexcess', Float64), ('bicarbonate', Float64), ('totalco2', Float64), ('carboxyhemoglobin', Float64), ('chloride', Float64), ('calcium', Float64), ('glucose', Float64), ('hematocrit', Float64), ('hemoglobin', Float64), ('intubated', Float64), ('lactate', Float64), ('methemoglobin', Float64), ('o2flow', Float64), ('fio2', Float64), ('so2', Float64), ('pco2', Float64), ('peep', Float64), ('ph', Float64), ('po2', Float64), ('potassium', Float64), ('requiredo2', Float64), ('sodium', Float64), ('temperature', Float64), ('tidalvolume', Float64), ('ventilationrate', Float64), ('ventilator', Float64), ('subject_id_right', Int64), ('hadm_id_right', Int64), ('

ventilator_setting.csv

In [1]:
#!/usr/bin/env python
# build_ventilator_setting.py
# ---------------------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path

# location of the raw CHARTEVENTS CSV
CE_CSV   = Path("./data/icu/chartevents.csv")   # change if needed
OUT_FILE = "ventilator_setting.csv"

# ──────────────────────────────────────────────────────────────
# 1 ▸ load only the itemids we care about
# ──────────────────────────────────────────────────────────────
ITEMS = [
    224688, 224689, 224690, 224687, 224685, 224684, 224686,
    224696, 220339, 224700, 223835, 223849, 229314, 223848, 224691
]

usecols = ["subject_id","stay_id","charttime","itemid",
           "value","valuenum","valueuom","storetime"]

ce = pd.read_csv(
        CE_CSV,
        usecols = usecols,
        parse_dates = ["charttime","storetime"]
)
# basic filters: non-null value and stay_id, item in list
ce = ce[
    ce["itemid"].isin(ITEMS) &
    ce["value"].notna() &
    ce["stay_id"].notna()
].copy()

# unify dtypes
ce = ce.astype({"stay_id":"Int32","itemid":"int64"})

# ──────────────────────────────────────────────────────────────
# 2 ▸ clean VALUENUM per SQL rules
# ──────────────────────────────────────────────────────────────
def clean_valuenum(row):
    iid, v = row.itemid, row.valuenum
    if iid == 223835:                         # FiO₂
        if 0.20 <= v <= 1:  return v*100          # fraction → %
        if 1 < v < 20:     return np.nan          # junk (flow rate)
        if 20 <= v <=100:  return v               # already %
        return np.nan
    if iid in (220339,224700):                # PEEP, keep 0-100
        if 0 <= v <= 100: return v
        return np.nan
    return v                                   # others unchanged

ce["valuenum"] = ce.apply(clean_valuenum, axis=1)

# ──────────────────────────────────────────────────────────────
# 3 ▸ keep the *latest* row per (subject_id, charttime, itemid)
#     (mimics ROW_NUMBER ORDER BY storetime DESC rn=1)
# ──────────────────────────────────────────────────────────────
ce.sort_values("storetime", ascending=False, inplace=True)
ce = ce.drop_duplicates(
        subset=["subject_id","charttime","itemid"],
        keep="first"
)

# ──────────────────────────────────────────────────────────────
# 4 ▸ aggregate to wide format
# ──────────────────────────────────────────────────────────────
mapping_valnum = {
    224688: "respiratory_rate_set",
    224690: "respiratory_rate_total",
    224689: "respiratory_rate_spontaneous",
    224687: "minute_volume",
    224684: "tidal_volume_set",
    224685: "tidal_volume_observed",
    224686: "tidal_volume_spontaneous",
    224696: "plateau_pressure",
    220339: "peep",                # one of two itemids
    224700: "peep",
    223835: "fio2",
    224691: "flow_rate",
}
mapping_value  = {
    223849: "ventilator_mode",
    229314: "ventilator_mode_hamilton",
    223848: "ventilator_type",
}

# split into numeric & string frames
num_df = ce[ce["itemid"].isin(mapping_valnum)].copy()
str_df = ce[ce["itemid"].isin(mapping_value)].copy()

num_df["column"] = num_df["itemid"].map(mapping_valnum)
str_df["column"] = str_df["itemid"].map(mapping_value)

# pivot numeric
num_wide = (
    num_df.pivot_table(
        index=["subject_id","charttime"],
        columns="column",
        values="valuenum",
        aggfunc="max"
    )
)

# pivot string
str_wide = (
    str_df.pivot_table(
        index=["subject_id","charttime"],
        columns="column",
        values="value",
        aggfunc="max"
    )
)

# combine & add stay_id = max(stay_id)
stay = ce.groupby(["subject_id","charttime"])["stay_id"].max()

wide = (
    pd.concat([stay, num_wide, str_wide], axis=1)
      .reset_index()
      .astype({"stay_id":"Int32"})
)

# reorder columns roughly as SQL output
col_order = ["subject_id","stay_id","charttime",
             "respiratory_rate_set","respiratory_rate_total",
             "respiratory_rate_spontaneous","minute_volume",
             "tidal_volume_set","tidal_volume_observed",
             "tidal_volume_spontaneous","plateau_pressure",
             "peep","fio2","flow_rate",
             "ventilator_mode","ventilator_mode_hamilton",
             "ventilator_type"]
wide = wide.reindex(columns=col_order)

# ──────────────────────────────────────────────────────────────
# 5 ▸ save
# ──────────────────────────────────────────────────────────────
wide.sort_values(["subject_id","charttime"]).to_csv(OUT_FILE, index=False)
print(f"Done – {len(wide):,} rows → {OUT_FILE}")

Done – 1,010,169 rows → ventilator_setting.csv


oxygen_delivery.csv

In [ ]:
#!/usr/bin/env python
# build_oxygen_delivery.py  –  recreate mimiciv_derived.oxygen_delivery
# --------------------------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path

ICU_PATH = Path("./data/icu/chartevents.csv")   # adjust if needed
OUTFILE  = "oxygen_delivery.csv"

# ──────────────────────────────────────────────────────────────
# 1 ▸ load only the itemids we need
# ──────────────────────────────────────────────────────────────
ITEMS_FLOW  = [223834, 227582, 227287]
ITEM_DEVICE = 226732

usecols = ["subject_id","stay_id","charttime","itemid",
           "value","valuenum","valueuom","storetime"]

ce = pd.read_csv(ICU_PATH, usecols=usecols, parse_dates=["charttime","storetime"])
ce = ce.astype({"stay_id":"Int32","itemid":"int64"})

# ──────────────────────────────────────────────────────────────
# 2 ▸ ce_stg2  – latest FLOW rows per (subject,charttime,itemid)
# ──────────────────────────────────────────────────────────────
flows = ce[ce["itemid"].isin(ITEMS_FLOW) & ce["value"].notna()].copy()

# map 227582 → 223834
flows.loc[flows["itemid"] == 227582, "itemid"] = 223834

flows.sort_values("storetime", ascending=False, inplace=True)
flows = flows.drop_duplicates(subset=["subject_id","charttime","itemid"], keep="first")

# ──────────────────────────────────────────────────────────────
# 3 ▸ o2  – device rows with row-number per charttime
# ──────────────────────────────────────────────────────────────
o2 = ce[ce["itemid"] == ITEM_DEVICE][["subject_id","stay_id","charttime","value"]].copy()
o2.rename(columns={"value":"o2_device"}, inplace=True)

# rank devices (NULL first, then alphabetic)
o2["rank_key"] = o2["o2_device"].isna()
o2.sort_values(["subject_id","charttime","rank_key","o2_device"], inplace=True)
o2["rn"] = (
    o2.groupby(["subject_id","charttime"]).cumcount() + 1
)                                              # 1,2,3,4 …

o2.drop(columns="rank_key", inplace=True)

# ──────────────────────────────────────────────────────────────
# 4 ▸ merge & aggregate to final view
# ──────────────────────────────────────────────────────────────
#   join on (subject_id, charttime)  ≈ FULL OUTER
merged = flows.merge(
    o2, on=["subject_id","charttime"], how="outer", suffixes=("","_o2")
)

# prefer stay_id from flows, else from o2
merged["stay_id"] = merged["stay_id"].fillna(merged["stay_id_o2"]).astype("Int32")
merged["rn"]      = merged["rn"].fillna(0).astype(int)

def agg_block(df):
    out = {
        "stay_id" : df["stay_id"].max(),
        "o2_flow" : df.loc[df["itemid"]==223834, "valuenum"].max(),
        "o2_flow_additional" : df.loc[df["itemid"]==227287, "valuenum"].max(),
    }
    # devices by rank (1..4)
    for k in range(1,5):
        col = f"o2_delivery_device_{k}"
        out[col] = df.loc[df["rn"]==k, "o2_device"].max()
    return pd.Series(out)

final = (
    merged.groupby(["subject_id","charttime"], as_index=False)
          .apply(agg_block)
)

# ──────────────────────────────────────────────────────────────
# 5 ▸ save
# ──────────────────────────────────────────────────────────────
final.to_csv(OUTFILE, index=False)
print(f"Done – {len(final):,} rows → {OUTFILE}")

Done – 1,503,989 rows → oxygen_delivery.csv


: 

ventilation.csv

In [4]:
#!/usr/bin/env python
# build_ventilation.py – derive ventilation durations from MIMIC-IV CSV
# ---------------------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path

DERIVED = Path("./data/icu")          # adjust if needed

# ─────────────────────────────────────────
# 1 ▸ load the two source views
# ─────────────────────────────────────────
vs  = pd.read_csv(
        DERIVED / "ventilator_setting.csv",
        usecols=["stay_id","charttime",
                 "ventilator_mode","ventilator_mode_hamilton"],
        parse_dates=["charttime"]
)
od  = pd.read_csv(
        DERIVED / "oxygen_delivery.csv",
        usecols=["stay_id","charttime","o2_delivery_device_1"],
        parse_dates=["charttime"]
)

# make sure types match
vs["stay_id"] = od["stay_id"] = vs["stay_id"].astype("int32")

# ─────────────────────────────────────────
# 2 ▸ tm  = union of timestamps
# ─────────────────────────────────────────
tm = (
    pd.concat([
        vs[["stay_id","charttime"]],
        od[["stay_id","charttime"]]
    ], ignore_index=True)
    .drop_duplicates()
)

# ─────────────────────────────────────────
# 3 ▸ join rows back to full context
# ─────────────────────────────────────────
full = (
    tm
    .merge(vs, how="left", on=["stay_id","charttime"])
    .merge(od, how="left", on=["stay_id","charttime"])
)

# ─────────────────────────────────────────
# 4 ▸ classify ventilation_status
# ─────────────────────────────────────────
trach  = {"Tracheostomy tube","Trach mask "}
inv_o2 = {"Endotracheal tube"}
inv_vm = {'(S) CMV','APRV','APRV/Biphasic+ApnPress','APRV/Biphasic+ApnVol',
          'APV (cmv)','Ambient','Apnea Ventilation','CMV','CMV/ASSIST',
          'CMV/ASSIST/AutoFlow','CMV/AutoFlow','CPAP/PPS','CPAP/PSV',
          'CPAP/PSV+Apn TCPL','CPAP/PSV+ApnPres','CPAP/PSV+ApnVol','MMV',
          'MMV/AutoFlow','MMV/PSV','MMV/PSV/AutoFlow','P-CMV','PCV+',
          'PCV+/PSV','PCV+Assist','PRES/AC','PRVC/AC','PRVC/SIMV','PSV/SBT',
          'SIMV','SIMV/AutoFlow','SIMV/PRES','SIMV/PSV','SIMV/PSV/AutoFlow',
          'SIMV/VOL','SYNCHRON MASTER','SYNCHRON SLAVE','VOL/AC'}
inv_ham = {'APRV','APV (cmv)','Ambient','(S) CMV','P-CMV','SIMV','APV (simv)',
           'P-SIMV','VS','ASV'}
niv_o2  = {'Bipap mask ','CPAP mask '}
niv_ham = {'DuoPaP','NIV','NIV-ST'}
hfnc    = {'High flow nasal cannula'}
supp_o2 = {'Non-rebreather','Face tent','Aerosol-cool','Venti mask ',
           'Medium conc mask ','Ultrasonic neb','Vapomist','Oxymizer',
           'High flow neb','Nasal cannula'}

def classify(row):
    dev = row.o2_delivery_device_1
    vm  = row.ventilator_mode
    vh  = row.ventilator_mode_hamilton
    if dev in trach:  return "Tracheostomy"
    if (dev in inv_o2) or (vm in inv_vm) or (vh in inv_ham): return "InvasiveVent"
    if (dev in niv_o2) or (vh in niv_ham):                   return "NonInvasiveVent"
    if dev in hfnc:     return "HFNC"
    if dev in supp_o2:  return "SupplementalOxygen"
    if dev == "None":   return "None"
    return np.nan

full["ventilation_status"] = full.apply(classify, axis=1)
full = full.dropna(subset=["ventilation_status"])

# ─────────────────────────────────────────
# 5 ▸ window calculations per stay
# ─────────────────────────────────────────
full = full.sort_values(["stay_id","charttime"])
grp  = full.groupby("stay_id")

full["charttime_lag"]           = grp["charttime"].shift(1)
full["charttime_lead"]          = grp["charttime"].shift(-1)
full["ventilation_status_lag"]  = grp["ventilation_status"].shift(1)

# gap (hours) between current row and previous row
full["gap_prev_hr"] = (
    (full["charttime"] - full["charttime_lag"])
    .dt.total_seconds()/3600
)

# new ventilation event?
full["new_event"] = (
    full["ventilation_status_lag"].isna()
    | (full["gap_prev_hr"] >= 14)
    | (full["ventilation_status_lag"] != full["ventilation_status"])
).astype(int)

# sequence id within stay
full["vent_seq"] = grp["new_event"].cumsum()

# end-time candidate for each row
gap_next_hr = (
    (full["charttime_lead"] - full["charttime"])
    .dt.total_seconds()/3600
)
full["end_candidate"] = np.where(
    full["charttime_lead"].isna() | (gap_next_hr >= 14),
    full["charttime"],
    full["charttime_lead"]
)

# ─────────────────────────────────────────
# 6 ▸ aggregate to intervals
# ─────────────────────────────────────────
out = (
    full.groupby(["stay_id","vent_seq"], as_index=False)
        .agg(
            starttime = ("charttime", "min"),
            endtime   = ("end_candidate","max"),
            ventilation_status = ("ventilation_status","last"),
        )
)

# discard single-timestamp intervals
out = out[out["starttime"] != out["endtime"]]

# save
out.to_csv("ventilation.csv", index=False)
print(f"Done – {len(out):,} ventilation intervals saved to ventilation.csv")

Done – 143,420 ventilation intervals saved to ventilation.csv


ventdurations.csv

In [ ]:
"""
----------------
Create ventdurations.csv from local MIMIC-IV files.

Input  (./data/)
    • icu/icustays.csv[.gz]          – stay_id, subject_id, hadm_id, intime
    • icu/ventilation.csv[.gz]       – start/end times of ventilation episodes
Output
    • icu/ventdurations.csv          – subject_id, hadm_id, stay_id, vent
"""

from pathlib import Path
import pandas as pd
from collections import defaultdict

# ----------------------------------------------------------------------
# 1.  Recursive loader
# ----------------------------------------------------------------------
ROOT = Path("./data")                

def load(name: str, *, columns=None, parse_dates=None) -> pd.DataFrame:
    """Return DataFrame for the first <name>.csv(.gz) found under ROOT."""
    files = list(ROOT.rglob(f"{name}.csv"))
    return pd.read_csv(files[0],
                       usecols=columns,
                       parse_dates=parse_dates,
                       low_memory=False)

# ----------------------------------------------------------------------
# 2.  ICU stays (gives us `intime`)
# ----------------------------------------------------------------------
print("Loading icustays …")
icu = load("icustays",
           columns=["stay_id", "subject_id", "hadm_id", "intime"],
           parse_dates=["intime"])
icu.set_index("stay_id", inplace=True)         # for O(1) lookup
intime_series = icu["intime"]

# ----------------------------------------------------------------------
# 3.  Scan ventilation episodes and flag qualifying stays
# ----------------------------------------------------------------------
print("Scanning ventilation table …")
vent_flag = defaultdict(int)
CHUNK = 500_000
columns = ["stay_id", "starttime", "endtime"]

vent_path = list(ROOT.rglob("ventilation.csv"))

vent_path = vent_path[0]

for chunk in pd.read_csv(vent_path,
                         usecols=columns,
                         parse_dates=["starttime", "endtime"],
                         chunksize=CHUNK,
                         low_memory=False):

    # keep rows belonging to stays we know
    chunk = chunk[chunk["stay_id"].isin(intime_series.index)]
    if chunk.empty:
        continue

    # attach ICU admit time
    chunk = chunk.join(intime_series, on="stay_id", how="left")

    # First-day overlap test:
    #   (start ≤ intime+24 h) AND (end ≥ intime)
    t0   = chunk["intime"]
    t24  = t0 + pd.Timedelta(hours=24)
    mask = (chunk["starttime"] <= t24) & (chunk["endtime"] >= t0)
    for stay in chunk.loc[mask, "stay_id"]:
        vent_flag[stay] = 1            # once flagged, always 1

# ----------------------------------------------------------------------
# 4.  Assemble result
# ----------------------------------------------------------------------
print("Building ventdurations.csv …")
out = (icu.reset_index()[["subject_id", "hadm_id", "stay_id"]]
          .assign(vent=lambda df: df["stay_id"].map(vent_flag).fillna(0).astype(int))
          .sort_values(["subject_id", "hadm_id", "stay_id"]))

out_path = ROOT / "icu" / "ventdurations.csv"
out.to_csv(out_path, index=False)
print(f"Wrote {len(out):,} rows ➜ {out_path}")

Loading icustays …
Scanning ventilation table …
Building ventdurations.csv …
Wrote 73,141 rows ➜ data/icu/ventdurations.csv


complete_blood_count.csv

In [ ]:
# build_complete_blood_count.py  –  with stay_id
# ---------------------------------------------------------------
import pandas as pd
from pathlib import Path

DATA      = Path("./data")          
LAB_CSV   = DATA / "hosp" / "labevents.csv"
ICU_CSV   = DATA / "icu"  / "icustays.csv"
OUT_CSV   = "complete_blood_count.csv"

# itemid → column name
ITEM_MAP = {
    51221: "hematocrit",
    51222: "hemoglobin",
    51248: "mch",
    51249: "mchc",
    51250: "mcv",
    51265: "platelet",
    51279: "rbc",
    51277: "rdw",
    51301: "wbc",
}

USE_LAB = ["specimen_id","subject_id","hadm_id",
           "charttime","itemid","valuenum"]
USE_ICU = ["subject_id","stay_id","intime","outtime"]

# ──────────────────────────────────────────────────────────────
# 1 ▸ load lab rows 
# ──────────────────────────────────────────────────────────────
le = (
    pd.read_csv(LAB_CSV, usecols=USE_LAB, parse_dates=["charttime"])
      .query("itemid in @ITEM_MAP.keys() and valuenum.notna() and valuenum>0")
)

# ──────────────────────────────────────────────────────────────
# 2 ▸ attach stay_id (charttime inside ICU window)
# ──────────────────────────────────────────────────────────────
icu = pd.read_csv(ICU_CSV, usecols=USE_ICU, parse_dates=["intime","outtime"])
icu = icu.astype({"stay_id":"int32"})

# merge (cartesian on subject_id) then filter by time window
le = (
    le.merge(icu, on="subject_id", how="left")
      .query("charttime >= intime and charttime <= outtime")
)

# keep 1-to-1 (if a lab falls into multiple overlapping stays, keep first)
le.sort_values(["specimen_id","stay_id"], inplace=True)
le = le.drop_duplicates(subset="specimen_id", keep="first")

# ──────────────────────────────────────────────────────────────
# 3 ▸ pivot MAX() per specimen_id
# ──────────────────────────────────────────────────────────────
pivot_vals = (
    le.pivot_table(index="specimen_id",
                   columns="itemid",
                   values="valuenum",
                   aggfunc="max")
      .rename(columns=ITEM_MAP)
)

ids = (
    le.groupby("specimen_id", as_index=True)
      .agg(subject_id=("subject_id","max"),
           hadm_id   =("hadm_id","max"),
           stay_id   =("stay_id","max"),
           charttime =("charttime","max"))
)

cbc = (
    ids.join(pivot_vals, how="left")
       .reset_index()
)

# ensure every expected analyte column exists
for col in ITEM_MAP.values():
    if col not in cbc:
        cbc[col] = pd.NA                     # or np.nan

# final column order
cbc = cbc.loc[:, ["subject_id","hadm_id","stay_id","charttime","specimen_id"]
                   + list(ITEM_MAP.values())] \
         .astype({"subject_id":"int32",
                  "hadm_id":"Int32",
                  "stay_id":"Int32",
                  "specimen_id":"int64"}) \
         .sort_values(["subject_id","charttime"])

# ──────────────────────────────────────────────────────────────
# 4 ▸ save
# ──────────────────────────────────────────────────────────────
cbc.to_csv(OUT_CSV, index=False)
print(f"Done – {len(cbc):,} rows → {OUT_CSV}")

Done – 437,065 rows → complete_blood_count.csv


chemistry.csv

In [ ]:
# build_chemistry.py  –  chemistry panel with stay_id
# ---------------------------------------------------
import pandas as pd
from pathlib import Path

DATA     = Path("./data")                      
LAB_CSV  = DATA / "hosp" / "labevents.csv"
ICU_CSV  = DATA / "icu"  / "icustays.csv"
OUT_CSV  = "chemistry.csv"

# itemid → (column, upper-limit, allow_zero)
CHEM = {
    50862: ("albumin"       ,   10 , False),
    # 50930: ("globulin"      ,   10 , False),
    50976: ("total_protein" ,   20 , False),
    50868: ("aniongap"      ,10000 ,  True),   # zero OK
    50882: ("bicarbonate"   ,10000 , False),
    51006: ("bun"           ,  300 , False),
    50893: ("calcium"       ,10000 , False),
    50902: ("chloride"      ,10000 , False),
    50912: ("creatinine"    ,  150 , False),
    50931: ("glucose"       ,10000 , False),
    50983: ("sodium"        ,  200 , False),
    50971: ("potassium"     ,   30 , False),
}

# ── columns to load ───────────────────────────────────────────
LAB_COLS = ["specimen_id","subject_id","hadm_id",
            "charttime","itemid","valuenum"]
ICU_COLS = ["subject_id","stay_id","intime","outtime"]

# ── 1 ▸ load and validate lab rows ────────────────────────────
lab = (
    pd.read_csv(LAB_CSV, usecols=LAB_COLS, parse_dates=["charttime"])
      .query("itemid in @CHEM.keys()")
)

def ok(row):
    _, lim, allow0 = CHEM[row.itemid]
    v = row.valuenum
    return pd.notna(v) and v >= 0 and (allow0 or v > 0) and v <= lim

lab = lab[lab.apply(ok, axis=1)].copy()

# ── 2 ▸ attach stay_id via merge + window filter ──────────────
icu = (
    pd.read_csv(ICU_CSV, usecols=ICU_COLS, parse_dates=["intime","outtime"])
      .astype({"stay_id":"int32"})
)

lab = (
    lab.merge(icu, on="subject_id", how="left")           # cartesian merge
       .query("charttime >= intime and charttime <= outtime")
)

lab.sort_values(["specimen_id","stay_id"], inplace=True)
lab = lab.drop_duplicates(subset="specimen_id", keep="first")

# ── 3 ▸ pivot MAX() per specimen_id ───────────────────────────
lab["col"] = lab["itemid"].map({k:v[0] for k,v in CHEM.items()})

vals = (
    lab.pivot_table(index="specimen_id",
                    columns="col",
                    values="valuenum",
                    aggfunc="max")
)

ids = (
    lab.groupby("specimen_id", as_index=True)
        .agg(subject_id=("subject_id","max"),
             hadm_id   =("hadm_id","max"),
             stay_id   =("stay_id","max"),
             charttime =("charttime","max"))
)

chem = ids.join(vals, how="left").reset_index()


cols = ["subject_id","hadm_id","stay_id","charttime","specimen_id"] \
       + [v[0] for v in CHEM.values()]

chem = (
    chem[cols]
        .astype({"subject_id":"int32",
                 "hadm_id"  :"Int32",
                 "stay_id"  :"Int32",
                 "specimen_id":"int64"})
        .sort_values(["subject_id","charttime"])
)

# ── 4 ▸ save ──────────────────────────────────────────────────
chem.to_csv(OUT_CSV, index=False)
print(f"Done – {len(chem):,} rows → {OUT_CSV}")

Done – 470,912 rows → chemistry.csv


blood_differential.csv

In [ ]:
#!/usr/bin/env python
# build_blood_differential.py  –  with stay_id (robust merge version)
# ------------------------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path

ROOT     = Path("./data")                     
LAB_CSV  = ROOT / "hosp" / "labevents.csv"
ICU_CSV  = ROOT / "icu"  / "icustays.csv"
OUT_CSV  = "blood_differential.csv"

# ── ITEMID → (column, factor) ─────────────────────────────────
ITEMS = {
    51300: ("wbc", 1.0), 51301: ("wbc", 1.0), 51755: ("wbc", 1.0),
    # 52069: ("basophils_abs", 1.0),
    52073: ("eosinophils_abs", 1.0), 51199: ("eosinophils_abs", 1/1000),
    51133: ("lymphocytes_abs", 1.0), 52769: ("lymphocytes_abs", 1/1000),
    52074: ("monocytes_abs", 1.0),   51253: ("monocytes_abs", 1/1000),
    52075: ("neutrophils_abs", 1.0),
    51218: ("granulocytes_abs", 1/1000),
    51146: ("basophils", 1.0),
    51200: ("eosinophils", 1.0),
    51244: ("lymphocytes", 1.0), 51245: ("lymphocytes", 1.0),
    51254: ("monocytes", 1.0),
    51256: ("neutrophils", 1.0),
    51143: ("atypical_lymphocytes", 1.0),
    51144: ("bands", 1.0),
    52135: ("immature_granulocytes", 1.0),
    51251: ("metamyelocytes", 1.0),
    51257: ("nrbc", 1.0),
}

LAB_COLS = ["specimen_id","subject_id","hadm_id",
            "charttime","itemid","valuenum"]
ICU_COLS = ["subject_id","stay_id","intime","outtime"]

# ──────────────────────────────────────────────────────────────
# 1 ▸ load & basic filter
# ──────────────────────────────────────────────────────────────
lab = (
    pd.read_csv(LAB_CSV, usecols=LAB_COLS, parse_dates=["charttime"])
      .query("itemid in @ITEMS.keys() and valuenum.notna() and valuenum>=0")
      .copy()
)

# convert units & set variable name
lab["var"] = lab["itemid"].map({k:v[0] for k,v in ITEMS.items()})
lab["val"] = lab.apply(lambda r: r.valuenum * ITEMS[r.itemid][1], axis=1)

# ──────────────────────────────────────────────────────────────
# 2 ▸ attach stay_id (merge + time-window filter)
# ──────────────────────────────────────────────────────────────
icu = (
    pd.read_csv(ICU_CSV, usecols=ICU_COLS, parse_dates=["intime","outtime"])
      .astype({"stay_id":"int32"})
)

lab = (
    lab.merge(icu, on="subject_id", how="left")                # cartesian
       .query("charttime >= intime and charttime <= outtime")  # within stay
)

# if a specimen overlaps multiple stays keep first
lab.sort_values(["specimen_id","stay_id"], inplace=True)
lab = lab.drop_duplicates("specimen_id", keep="first")

# ──────────────────────────────────────────────────────────────
# 3 ▸ pivot MAX() per specimen_id
# ──────────────────────────────────────────────────────────────
vals = (
    lab.pivot_table(index="specimen_id",
                    columns="var",
                    values="val",
                    aggfunc="max")
)

ids = (
    lab.groupby("specimen_id", as_index=True)
        .agg(subject_id=("subject_id","max"),
             hadm_id   =("hadm_id","max"),
             stay_id   =("stay_id","max"),
             charttime =("charttime","max"))
)

diff = ids.join(vals, how="left").reset_index()

# ──────────────────────────────────────────────────────────────
# 4 ▸ impute *_abs when possible
# ──────────────────────────────────────────────────────────────
pct_cols = ["basophils", "eosinophils", "lymphocytes",
            "monocytes", "neutrophils"]
abs_cols = [c + "_abs" for c in pct_cols]

# make sure every absolute/percent column exists
for col in abs_cols + pct_cols:
    if col not in diff.columns:
        diff[col] = pd.NA          # create the column filled with <NA>

diff["impute_abs"] = (
    (diff["wbc"].fillna(0) > 0) &
    diff[pct_cols].fillna(0).sum(axis=1).gt(0)
)

for pct, abs_ in zip(pct_cols, abs_cols):
    mask = diff[abs_].isna() & diff[pct].notna() & diff["impute_abs"]
    diff.loc[mask, abs_] = diff.loc[mask, pct] * diff.loc[mask, "wbc"]

diff[abs_cols] = diff[abs_cols].round(4)

# ──────────────────────────────────────────────────────────────
# 5 ▸ order & save
# ──────────────────────────────────────────────────────────────
final_cols = (["subject_id","hadm_id","stay_id","charttime","specimen_id","wbc"]
              + abs_cols
              + pct_cols
              + ["atypical_lymphocytes","bands","immature_granulocytes",
                 "metamyelocytes","nrbc"])

available_cols = [col for col in final_cols if col in diff.columns]
diff = diff[available_cols].astype({
    "subject_id": "int32",
    "hadm_id": "Int32",
    "stay_id": "int32"
})

diff.to_csv(OUT_CSV, index=False)
print(f"Done – {len(diff):,} rows → {OUT_CSV}")

/var/folders/cm/blwm27yd02l6xpvwrm9_l41m0000gn/T/ipykernel_70599/1088665163.py:107: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  diff.loc[mask, abs_] = diff.loc[mask, pct] * diff.loc[mask, "wbc"]


Done – 376,749 rows → blood_differential.csv


coagulation.csv

In [ ]:
#!/usr/bin/env python
# build_coagulation.py  –  with stay_id
# -------------------------------------------------------------
import pandas as pd
from pathlib import Path

ROOT     = Path("./data")                      
LAB_CSV  = ROOT / "hosp" / "labevents.csv"
ICU_CSV  = ROOT / "icu"  / "icustays.csv"
OUT_CSV  = "coagulation.csv"

# itemid → column name
ITEMS = {
    51196: "d_dimer",
    51214: "fibrinogen",
    51297: "thrombin",
    51237: "inr",
    51274: "pt",
    51275: "ptt",
}

LAB_COLS = ["specimen_id","subject_id","hadm_id",
            "charttime","itemid","valuenum"]

# ── 1. load & filter labs ────────────────────────────────────
labs = (
    pd.read_csv(LAB_CSV, usecols=LAB_COLS, parse_dates=["charttime"])
      .query("itemid in @ITEMS.keys() and valuenum.notna()")
      .copy()
)

# ── 2. load ICU stays & build IntervalIndex for fast lookup ──
icu = (
    pd.read_csv(ICU_CSV,
                usecols=["subject_id","stay_id","intime","outtime"],
                parse_dates=["intime","outtime"])
      .astype({"stay_id":"int32"})
)

ivl = pd.IntervalIndex.from_arrays(icu["intime"], icu["outtime"], closed="both")
icu_int = icu.set_index(ivl)


def match_stay(row):
    hits = icu_int[icu_int.index.contains(row.charttime)]
    hits = hits[hits["subject_id"] == row.subject_id]
    return hits["stay_id"].iloc[0] if not hits.empty else pd.NA

labs["stay_id"] = labs.apply(match_stay, axis=1)

# ── 3. pivot MAX() per specimen_id ────────────────────────────
labs["var"] = labs["itemid"].map(ITEMS)

vals = (
    labs.pivot_table(index="specimen_id",
                     columns="var",
                     values="valuenum",
                     aggfunc="max")
)

ids = (
    labs.groupby("specimen_id", as_index=True)
        .agg(subject_id=("subject_id","max"),
             hadm_id   =("hadm_id","max"),
             stay_id   =("stay_id","max"),        # may stay <NA>
             charttime =("charttime","max"))
)

coag = (
    ids.join(vals, how="left")
       .reset_index()
       .loc[:, ["subject_id","hadm_id","stay_id","charttime","specimen_id"]
               + list(ITEMS.values())]
       .astype({"subject_id":"int32",
                "hadm_id"  :"Int32",
                "stay_id"  :"Int32",
                "specimen_id":"int64"})
       .sort_values(["subject_id","charttime"])
)

# ── 4. save ──────────────────────────────────────────────────
coag.to_csv(OUT_CSV, index=False)
print(f"Done – {len(coag):,} rows → {OUT_CSV}")

Done – 1,545,930 rows → coagulation.csv


enzyme.csv

In [30]:
#!/usr/bin/env python
# build_enzyme.py  –  enzymes & bilirubin panel with stay_id
# ----------------------------------------------------------------
import pandas as pd
from pathlib import Path

ROOT     = Path("./data")                       # change if needed
LAB_CSV  = ROOT / "hosp" / "labevents.csv"
ICU_CSV  = ROOT / "icu"  / "icustays.csv"
OUT_CSV  = "enzyme.csv"

# itemid → output column
ITEMS = {
    50861: "alt",    50863: "alp",    50878: "ast",
    50867: "amylase",
    50885: "bilirubin_total", 50883: "bilirubin_direct",
    50884: "bilirubin_indirect",
    50910: "ck_cpk", 50911: "ck_mb",
    50927: "ggt",    50954: "ld_ldh",
}

LAB_COLS = ["specimen_id","subject_id","hadm_id",
            "charttime","itemid","valuenum"]

# ──────────────────────────────────────────────────────────────
# 1 ▸ load & filter labs
# ──────────────────────────────────────────────────────────────
lab = (
    pd.read_csv(LAB_CSV, usecols=LAB_COLS, parse_dates=["charttime"])
      .query("itemid in @ITEMS.keys() and valuenum.notna() and valuenum>0")
      .copy()
)

# ──────────────────────────────────────────────────────────────
# 2 ▸ attach stay_id via ICU interval match
# ──────────────────────────────────────────────────────────────
icu = (
    pd.read_csv(ICU_CSV,
                usecols=["subject_id","stay_id","intime","outtime"],
                parse_dates=["intime","outtime"])
      .astype({"stay_id":"int32"})
)

# build IntervalIndex for fast lookup
ivl = pd.IntervalIndex.from_arrays(icu["intime"], icu["outtime"], closed="both")
icu_int = icu.set_index(ivl)



def find_stay(row):
    hits = icu_int[icu_int.index.contains(row.charttime)]
    hits = hits[hits["subject_id"] == row.subject_id]
    return hits["stay_id"].iloc[0] if not hits.empty else pd.NA

lab["stay_id"] = lab.apply(find_stay, axis=1)

# ──────────────────────────────────────────────────────────────
# 3 ▸ pivot MAX() per specimen_id
# ──────────────────────────────────────────────────────────────
lab["col"] = lab["itemid"].map(ITEMS)

vals = (
    lab.pivot_table(index="specimen_id",
                    columns="col",
                    values="valuenum",
                    aggfunc="max")
)

ids = (
    lab.groupby("specimen_id", as_index=True)
        .agg(subject_id=("subject_id","max"),
             hadm_id   =("hadm_id","max"),
             stay_id   =("stay_id","max"),   # may be <NA>
             charttime =("charttime","max"))
)

enzyme = ids.join(vals, how="left").reset_index()

# guarantee every analyte column exists
for col in ITEMS.values():
    if col not in enzyme.columns:
        enzyme[col] = pd.NA

cols_order = ["subject_id","hadm_id","stay_id","charttime","specimen_id"] \
             + list(ITEMS.values())

enzyme = (
    enzyme[cols_order]
        .astype({"subject_id":"int32",
                 "hadm_id"  :"Int32",
                 "stay_id"  :"Int32",
                 "specimen_id":"int64"})
        .sort_values(["subject_id","charttime"])
)

# ──────────────────────────────────────────────────────────────
# 4 ▸ save
# ──────────────────────────────────────────────────────────────
enzyme.to_csv(OUT_CSV, index=False)
print(f"Done – {len(enzyme):,} rows → {OUT_CSV}")

Done – 1,637,082 rows → enzyme.csv


first_day_lab.csv

In [ ]:
#!/usr/bin/env python
# build_first_day_lab.py  –  MIMIC-IV day-1 laboratory panel
# -----------------------------------------------------------
import pandas as pd
from pathlib import Path

ROOT = Path("./data")            
ICU  = ROOT / "icu"
DRV  = ROOT / "derived"         

OUT_CSV = "first_day_lab.csv"

# ──────────────────────────────────────────────────────────────
# 1 ▸ ICU stays  (need stay_id, subject_id, intime)            │
# ──────────────────────────────────────────────────────────────
icu = (
    pd.read_csv(ICU / "icustays.csv",
                usecols=["subject_id","stay_id","intime"],
                parse_dates=["intime"])
      .astype({"stay_id":"int32"})
)

icu_lookup = icu[["stay_id","intime"]]        # for time-window filter

def within_day(df: pd.DataFrame, time_col: str) -> pd.DataFrame:
    """
    Keep rows whose *time_col* lies in [intime −6 h … intime +24 h].
    Assumes df has a `stay_id` column; joins icu_lookup internally.
    """
    d = df.merge(icu_lookup, on="stay_id", how="left", validate="many_to_one")
    lo = d["intime"] - pd.Timedelta(hours=6)
    hi = d["intime"] + pd.Timedelta(days=1)
    keep = (d[time_col] >= lo) & (d[time_col] <= hi)
    return d.loc[keep].drop(columns="intime")

# ──────────────────────────────────────────────────────────────
# 2 ▸ helper to build min/max blocks                            │
# ──────────────────────────────────────────────────────────────
def build_block(csv: Path, cols_map: dict[str, str], time_col="charttime"):
    """
    csv       : derived table path
    cols_map  : { column_in_csv : short_name_in_output }
    returns   : dataframe indexed by stay_id with *_min/_max columns
    """
    usecols = ["stay_id", time_col] + list(cols_map.keys())
    df = pd.read_csv(csv, usecols=usecols, parse_dates=[time_col])
    # Convert stay_id to a nullable integer type to handle non-finite values.
    df["stay_id"] = pd.to_numeric(df["stay_id"], errors="coerce").astype("Int32")

    df = within_day(df, time_col)

    agg = {}
    for col, short in cols_map.items():
        agg[f"{short}_min"] = (col, "min")
        agg[f"{short}_max"] = (col, "max")

    df = (
        df.groupby("stay_id", as_index=False)
          .agg(**agg)
    )
    return df

# ──────────────────────────────────────────────────────────────
# 3 ▸ blocks following SQL CTEs                                │
# ──────────────────────────────────────────────────────────────
cbc_cols  = {
    "hematocrit" : "hematocrit",
    "hemoglobin" : "hemoglobin",
    "platelet"   : "platelets",
    "wbc"        : "wbc",
}
chem_cols = {
    "albumin":"albumin","total_protein":"total_protein",
    "aniongap":"aniongap","bicarbonate":"bicarbonate","bun":"bun",
    "calcium":"calcium","chloride":"chloride","creatinine":"creatinine",
    "glucose":"glucose","sodium":"sodium","potassium":"potassium"
}
diff_cols = {
    "abs_basophils":"abs_basophils","abs_eosinophils":"abs_eosinophils",
    "abs_lymphocytes":"abs_lymphocytes","abs_monocytes":"abs_monocytes",
    "abs_neutrophils":"abs_neutrophils","atyps":"atyps","bands":"bands",
    "imm_granulocytes":"imm_granulocytes","metas":"metas","nrbc":"nrbc"
}
coag_cols = {
    "d_dimer":"d_dimer","fibrinogen":"fibrinogen","thrombin":"thrombin",
    "inr":"inr","pt":"pt","ptt":"ptt"
}
enz_cols  = {
    "alt":"alt","alp":"alp","ast":"ast","amylase":"amylase",
    "bilirubin_total":"bilirubin_total",
    "bilirubin_direct":"bilirubin_direct",
    "bilirubin_indirect":"bilirubin_indirect",
    "ck_cpk":"ck_cpk","ck_mb":"ck_mb","ggt":"ggt","ld_ldh":"ld_ldh"
}

cbc  = build_block(DRV / "complete_blood_count.csv",    cbc_cols)
chem = build_block(DRV / "chemistry.csv",               chem_cols)
diff = build_block(
    DRV / "blood_differential.csv",
    {
        "wbc": "wbc",
        "basophils": "basophils",
        "eosinophils": "eosinophils",
        "lymphocytes": "lymphocytes",
        "monocytes": "monocytes",
        "neutrophils": "neutrophils",
        "atypical_lymphocytes": "atypical_lymphocytes",
        "bands": "bands",
        "nrbc": "nrbc"
    }
)
coag = build_block(DRV / "coagulation.csv", coag_cols)
coag["stay_id"] = pd.to_numeric(coag["stay_id"], errors="coerce").astype("Int32")
enz  = build_block(DRV / "enzyme.csv", enz_cols)
enz["stay_id"] = pd.to_numeric(enz["stay_id"], errors="coerce").astype("Int32")

# ──────────────────────────────────────────────────────────────
# 4 ▸ merge everything & attach subject_id                     │
# ──────────────────────────────────────────────────────────────
lab = (
    icu[["subject_id","stay_id"]]
      .merge(cbc,  on="stay_id", how="left")
      .merge(chem, on="stay_id", how="left")
      .merge(diff, on="stay_id", how="left")
      .merge(coag, on="stay_id", how="left")
      .merge(enz,  on="stay_id", how="left")
)

# ──────────────────────────────────────────────────────────────
# 5 ▸ save                                                     │
# ──────────────────────────────────────────────────────────────
lab.to_csv(OUT_CSV, index=False)
print(f"Done – {len(lab):,} rows → {OUT_CSV}")

Done – 73,141 rows → first_day_lab.csv


sofa.csv

In [ ]:
# build_first_day_sofa.py  – 
# -----------------------------------------------------------------
import pandas as pd, numpy as np
from pathlib import Path

BASE = Path("./data")                   
ICU  = BASE / "icu"                    
DRV  = BASE / "derived"                  
# helper ----------------------------------------------------------
def clip_by_intime(df, time_col, lookup, start_h=-6, end_h=24):
    """
    Keep rows whose time_col is within [intime+start_h … intime+end_h].
    lookup must have columns 'stay_id' and 'intime'.
    """
    df = df.merge(
        lookup,
        on="stay_id",
        how="left",
        suffixes=("", "_icu"),          # avoid _x/_y randomness
        validate="many_to_one",
    )

    # choose which column holds the admission time
    if "intime" in df.columns:
        icu_in = df["intime"].fillna(df["intime_icu"])
    else:                               # df had none → use the one from lookup
        icu_in = df.pop("intime_icu")

    lo = icu_in + pd.Timedelta(hours=start_h)
    hi = icu_in + pd.Timedelta(hours=end_h)

    keep = (df[time_col] >= lo) & (df[time_col] <= hi)
    df = df.loc[keep]

    # tidy up extra column if it exists
    df = df.drop(columns=[c for c in ["intime", "intime_icu"] if c in df])

    return df.reset_index(drop=True)

# 1 ─ ICU rows ----------------------------------------------------
icu = (
    pd.read_csv(ICU / "icustays.csv", parse_dates=["intime"])
      .loc[:, ["subject_id","hadm_id","stay_id","intime"]]
)
icu_lookup = icu.copy()                     # stay_id + intime
icu_lookup = icu[["stay_id","intime"]]        # for clip_by_intime
icu.set_index("stay_id", inplace=True)        # rest of the script unchanged

# 2 ─ vasopressor max rates (derived tables) ----------------------
VASO_ITEMID = {
    221906: "norepinephrine",
    30047 : "norepinephrine",
    221289: "epinephrine",
    221653: "dopamine",
    221286: "dobutamine",
}

def within_window(df, key, intime, start_h=-6, end_h=24):
    """Keep rows whose key-time lies in [intime+start_h, intime+end_h]."""
    lo = intime + pd.Timedelta(hours=start_h)
    hi = intime + pd.Timedelta(hours=end_h)
    return df[(df[key] >= lo) & (df[key] <= hi)]

inp = pd.read_csv(
        ICU / "inputevents.csv",
        usecols=["stay_id", "starttime", "itemid", "rate"],
        parse_dates=["starttime"],
    )
inp = inp[inp["itemid"].isin(VASO_ITEMID)]
inp["drug"] = inp["itemid"].map(VASO_ITEMID)
inp["stay_id"] = inp["stay_id"].astype("int32")

# keep only rows within -6 h … +24 h of ICU admission
inp = (
    inp.join(icu["intime"], on="stay_id", how="left")
       .pipe(lambda d: within_window(d, "starttime", d["intime"]))
)

vaso_max = (
    inp.groupby(["stay_id", "drug"], as_index=False)["rate"]
       .max()
       .pivot(index="stay_id", columns="drug", values="rate")
       .rename(columns=lambda c: f"rate_{c}")
)

# 3 ─ PaO2/FiO2 minima, with & w/o ventilation -------------------
bg  = pd.read_csv(DRV / "first_day_bg_art.csv",
                  usecols=["subject_id","charttime","PaO2FiO2","stay_id"],
                  parse_dates=["charttime"])
bg  = bg.merge(icu["intime"], left_on="stay_id", right_index=True, how="left")


bg  = clip_by_intime(bg, "charttime", icu_lookup)
inp = clip_by_intime(inp, "starttime", icu_lookup)

vent = pd.read_csv(DRV / "ventilation.csv",
                   usecols=["stay_id","starttime","endtime","ventilation_status"],
                   parse_dates=["starttime","endtime"])
vent = vent[vent["ventilation_status"]=="InvasiveVent"]

# mark BG rows that fall inside an invasive-vent interval
vent_grp = vent.groupby("stay_id")
def is_vent_row(row):
    v = vent_grp.get_group(row.stay_id) if row.stay_id in vent_grp.groups else None
    return 0 if v is None else int(((v.starttime<=row.charttime)&(row.charttime<=v.endtime)).any())

bg["isvent"] = bg.apply(is_vent_row, axis=1)

pafi = (
    bg.groupby("stay_id")
      .agg(
        pao2fio2_novent_min = ("PaO2FiO2",
                               lambda s: s[bg.loc[s.index,"isvent"]==0].min()),
        pao2fio2_vent_min   = ("PaO2FiO2",
                               lambda s: s[bg.loc[s.index,"isvent"]==1].min()),
      )
)

# 4 ─ first-day vitals / labs / urine / GCS -----------------------
vital = pd.read_csv(DRV / "first_day_vitalsign.csv",
                    usecols=["stay_id"]).set_index("stay_id")
lab   = pd.read_csv(DRV / "first_day_lab.csv",
                    usecols=["stay_id","creatinine_max",
                             "bilirubin_total_max","platelets_min"]
                   ).set_index("stay_id")
uo    = pd.read_csv(DRV / "first_day_urine_output.csv",
                    usecols=["stay_id","urineoutput"]
                   ).set_index("stay_id")
gcs   = pd.read_csv(DRV / "first_day_gcs.csv",
                    usecols=["stay_id","gcs_min"]
                   ).set_index("stay_id")

# 5 ─ gather all components --------------------------------------
comp = (
    icu.join([vaso_max, pafi, vital, lab, uo, gcs], how="left")
      .reset_index()
)

for drug in ["norepinephrine","epinephrine","dopamine","dobutamine"]:
    comp[f"rate_{drug}"] = comp[f"rate_{drug}"].astype(float)

# 6 ─ component score functions ----------------------------------
def resp(r):
    if pd.isna(r.pao2fio2_vent_min) and pd.isna(r.pao2fio2_novent_min): return np.nan
    if r.pao2fio2_vent_min   < 100: return 4
    if r.pao2fio2_vent_min   < 200: return 3
    if r.pao2fio2_novent_min < 300: return 2
    if r.pao2fio2_novent_min < 400: return 1
    return 0

def coag(p):
    if pd.isna(p): return np.nan
    if p<20: return 4
    if p<50: return 3
    if p<100: return 2
    if p<150: return 1
    return 0

def liver(b):
    if pd.isna(b): return np.nan
    if b>=12: return 4
    if b>=6:  return 3
    if b>=2:  return 2
    if b>=1.2:return 1
    return 0

def cardio(row):
    dopa,dobu,epi,norepi = (row.rate_dopamine,row.rate_dobutamine,
                                row.rate_epinephrine,row.rate_norepinephrine
                                )
    if (dopa>15) or (epi>0.1) or (norepi>0.1): return 4
    if (dopa>5) or (0<=epi<=0.1) or (0<=norepi<=0.1): return 3
    if (dopa>0) or (dobu>0): return 2
    if all(pd.isna(x) for x in [dopa,dobu,epi,norepi]): return np.nan
    return 0

def cns(g):
    if pd.isna(g): return np.nan
    if g<6: return 4
    if 6<=g<=9: return 3
    if 10<=g<=12: return 2
    if 13<=g<=14: return 1
    return 0

def renal(row):
    cr,uo = row.creatinine_max,row.urineoutput
    if (cr>=5) or (uo<200): return 4
    if (3.5<=cr<5) or (uo<500): return 3
    if 2<=cr<3.5: return 2
    if 1.2<=cr<2: return 1
    if pd.isna(cr) and pd.isna(uo): return np.nan
    return 0

# 7 ─ compute scores ---------------------------------------------
comp["respiration"]   = comp.apply(resp,  axis=1)
comp["coagulation"]   = comp.platelets_min.map(coag)
comp["liver"]         = comp.bilirubin_total_max.map(liver)
comp["cardiovascular"]= comp.apply(cardio, axis=1)
comp["cns"]           = comp.gcs_min.map(cns)
comp["renal"]         = comp.apply(renal,  axis=1)

cols = ["respiration","coagulation","liver","cardiovascular","cns","renal"]
comp["sofa"] = comp[cols].fillna(0).sum(axis=1)

# 8 ─ final output -----------------------------------------------
out = comp[["subject_id","hadm_id","stay_id","sofa"]+cols]
out.to_csv("first_day_sofa.csv", index=False)
print(f"Done – {len(out):,} rows → first_day_sofa.csv")

ValueError: Usecols do not match columns, columns expected but not found: ['urineoutput']

In [ ]:
df = pd.read_csv("./data/icu/first_day_vitalsign.csv", nrows=5)

print(df.head())

   subject_id   hadm_id   stay_id  heartrate_min  heartrate_max  \
0    12466550  23998182  30000153           83.0          128.0   
1    13180007  27543152  30000213           66.0           91.0   
2    12207593  22795209  30000646           69.0          102.0   
3    12980335  23552849  30001148           64.0           80.0   
4    12168737  29283664  30001336           53.0           73.0   

   heartrate_mean  sysbp_min  sysbp_max  sysbp_mean  diasbp_min  ...  \
0      106.840000      108.0      171.0  136.088235        55.0  ...   
1       81.680000      116.0      168.0  134.920000        47.0  ...   
2       86.423729       71.0      151.0   92.703704        34.0  ...   
3       75.520000       92.0      125.0  107.900000        48.0  ...   
4       63.040000      100.0      120.0  110.625000        47.0  ...   

   resprate_mean  tempc_min  tempc_max  tempc_mean  spo2_min  spo2_max  \
0      14.923077  37.222222  38.222222   37.500000      92.0     100.0   
1      19.437500